#**Lets create an ASR Model then try to read out how many audio files are presented**

In [1]:
! pip install git+https://github.com/openai/whisper.git -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 31.6 MB/s eta 0:00:00


In [2]:
!pip install SpeechRecognition ffmpeg-python jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 106.0 MB/s eta 0:00:00


In [3]:
import whisper
model = whisper.load_model("large") #769M Parameter

100%|██████████████████████████████████████| 2.88G/2.88G [00:24<00:00, 128MiB/s]


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
pip install --upgrade --no-deps --force-reinstall git+https://github.com/sanchit-gandhi/whisper-jax.git

  Cloning https://github.com/sanchit-gandhi/whisper-jax.git to /tmp/pip-req-build-w6351k9y
  Running command git clone --filter=blob:none --quiet https://github.com/sanchit-gandhi/whisper-jax.git /tmp/pip-req-build-w6351k9y
  Resolved https://github.com/sanchit-gandhi/whisper-jax.git to commit f983178a80ad37cf2f655777c26a74438b5d8690
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for whisper_jax: filename=whisper_jax-0.0.1-py3-none-any.whl size=60260 sha256=3c90d3e177373d64039b493592e36d1a406bf337424dbdabdb7be56ec56fe596
  Stored in directory: /tmp/pip-ephem-wheel-cache-_th4f9ki/wheels/19/7f/50/36e503232cafa814d4b088fcb9ce13d5d3b3c9e9d29e139a94
Successfully built whisper_jax


In [6]:
pip install cached_property

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
whisper-jax 0.0.1 requires transformers<4.35.0,>=4.27.4, but you have transformers 4.40.0 which is incompatible.


In [7]:
from whisper_jax import FlaxWhisperPipline
import jax.numpy as jnp
import requests

pipeline = FlaxWhisperPipline('parthiv11/indic_whisper_nodcil', dtype=jnp.bfloat16)

def transcribe_audio_from_url(audio_url):
    try:
        audio_response = requests.get(audio_url)
        audio_file_path = 'temp_audio.mp3'
        with open(audio_file_path, 'wb') as f:
            f.write(audio_response.content)

        transcript = pipeline(audio_file_path)

        import os
        os.remove(audio_file_path)

        return transcript
    except Exception as e:
        return f"Error: {str(e)}"



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.5k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

...

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

flax_model.msgpack:   0%|          | 0.00/1.53G [00:00<?, ?B/s]

Some of the weights of FlaxWhisperForConditionalGeneration were initialized in float16 precision from the model checkpoint at parthiv11/indic_whisper_nodcil:
[('model', 'decoder', 'embed_positions', 'embedding'), ('model', 'decoder', 'embed_tokens', 'embedding'), ('model', 'decoder', 'layer_norm', 'bias'), ('model', 'decoder', 'layer_norm', 'scale'), ('model', 'decoder', 'layers', '0', 'encoder_attn', 'k_proj', 'kernel'), ('model', 'decoder', 'layers', '0', 'encoder_attn', 'out_proj', 'bias'), ('model', 'decoder', 'layers', '0', 'encoder_attn', 'out_proj', 'kernel'), ('model', 'decoder', 'layers', '0', 'encoder_attn', 'q_proj', 'bias'), ('model', 'decoder', 'layers', '0', 'encoder_attn', 'q_proj', 'kernel'), ('model', 'decoder', 'layers', '0', 'encoder_attn', 'v_proj', 'bias'), ('model', 'decoder', 'layers', '0', 'encoder_attn', 'v_proj', 'kernel'), ('model', 'decoder', 'layers', '0', 'encoder_attn_layer_norm', 'bias'), ('model', 'decoder', 'layers', '0', 'encoder_attn_layer_norm', 'sc

generation_config.json:   0%|          | 0.00/3.49k [00:00<?, ?B/s]

In [8]:
import os

def list_audio_files(folder_path):
    audio_files = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.mp3') or filename.endswith('.wav'):
            audio_files.append(filename)
    return audio_files

folder_path = '/content/drive/MyDrive/GV_Eval_3h/Audio'

audio_files = list_audio_files(folder_path)

for audio_file in audio_files:
    print(audio_file)


01-08232-01.mp3
01-11375-01.mp3
02-18014-01.mp3
02-20927-01.mp3
02-18071-01.mp3
02-15302-01.mp3
01-09100-01.mp3
02-17067-01.mp3
02-18353-01.mp3
01-01227-01.mp3
02-16015-01.mp3
01-07119-03.mp3
01-02981-02.mp3
01-08042-01.mp3
02-13479-01.mp3
01-03128-03.mp3
01-06535-01.mp3
02-12566-02.mp3
02-17524-01.mp3
01-04281-03.mp3
02-19301-01.mp3
02-19443-01.mp3
02-18845-01.mp3
01-04318-01.mp3
02-17264-01.mp3
02-12848-02.mp3
01-02781-01.mp3
02-20406-01.mp3
01-11046-03.mp3
01-09592-02.mp3
02-13440-01.mp3
01-02757-02.mp3
02-15827-01.mp3
02-17346-01.mp3
02-13660-01.mp3
02-20288-01.mp3
01-09716-01.mp3
01-02279-03.mp3
01-10863-01.mp3
01-00703-03.mp3
01-08414-01.mp3
01-08615-02.mp3
01-00759-03.mp3
01-03682-02.mp3
01-11769-01.mp3
01-01782-01.mp3
01-11494-02.mp3
13-00314-01.mp3
01-10319-01.mp3
01-00262-02.mp3
01-07706-02.mp3
02-17956-01.mp3
01-09395-01.mp3
01-03031-02.mp3
02-19520-01.mp3
01-02956-01.mp3
01-08678-01.mp3
01-10827-03.mp3
01-00078-01.mp3
02-21579-01.mp3
01-00097-03.mp3
01-11389-01.mp3
01-05290

In [9]:
from IPython.display import Audio, display


display(Audio('/content/drive/MyDrive/GV_Eval_3h/Audio/01-00071-02.mp3', autoplay=True))

In [10]:
result = model.transcribe("/content/drive/MyDrive/GV_Eval_3h/Audio/01-00071-02.mp3",language='hi')
print(result["text"])

 नाले जान है जिससे निकला दंदा पाने सडको पजमा है, सिस्टा पाइल लेकर किसी प्रकार की कयावत होती भी नहीं कितितिया है।


In [11]:
!wget -O audio.mp3 http://www.moviesoundclips.net/movies1/batmanbegins/bats.mp3

--2024-04-25 09:34:53--  http://www.moviesoundclips.net/movies1/batmanbegins/bats.mp3
Resolving www.moviesoundclips.net (www.moviesoundclips.net)... 198.54.115.219
Connecting to www.moviesoundclips.net (www.moviesoundclips.net)|198.54.115.219|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 52128 (51K) [audio/mpeg]
Saving to: ‘audio.mp3’

audio.mp3           100%[===================>]  50.91K   144KB/s    in 0.4s    

2024-04-25 09:34:54 (144 KB/s) - ‘audio.mp3’ saved [52128/52128]



In [12]:
import glob

directory = '/content/drive/MyDrive/GV_Eval_3h/Audio'

mp3_pattern = '*.mp3'
wav_pattern = '*.wav'

mp3_files = glob.glob(directory + '**/' + mp3_pattern, recursive=True)
wav_files = glob.glob(directory + '**/' + wav_pattern, recursive=True)

audio_files = mp3_files + wav_files

for audio_file in audio_files:
    result = model.transcribe(audio_file, language='hi')
    print(audio_file, result["text"])


/content/drive/MyDrive/GV_Eval_3h/Audio/01-08232-01.mp3  कटिहार रेलखंड के मानसी चैदा बन्नी के बीच रेलपत्री क्रैक करने से 20 मिनट तक ट्रेन परिचालन बादित रहा
/content/drive/MyDrive/GV_Eval_3h/Audio/01-11375-01.mp3  आर्थिक रूप से मदद करने के लिए आप अपने भारतिय स्टेट बैंक के नज़दी की बैंक शाकार
/content/drive/MyDrive/GV_Eval_3h/Audio/02-18014-01.mp3  ब्रिटेन चीन मैच से होगा महिला चैंपिन्स ट्रॉफिय आगाज खबर है जांगजू से अंतरास्त्य होकी महसंग ने शुक्रवार को महिला होकी चैंपिन्स के शुरुआत ब्रिटेन चीन मैच से होगा
/content/drive/MyDrive/GV_Eval_3h/Audio/02-20927-01.mp3  पत्ना उच्छने आले ने जेल वाडन की 3000 पदों के लिए चल रही सारिरिक जाच परिख्षा को रोकने से इंकार कर दिया है।
/content/drive/MyDrive/GV_Eval_3h/Audio/02-18071-01.mp3  से हम लोगों को वेदन नहीं दिया जा रहा है इधर जिस आवास में हम लोग रहते हैं उसके मालिक को भी कंपनी की ओर से ताला बंद करने का फर्मान दिया गया है
/content/drive/MyDrive/GV_Eval_3h/Audio/02-15302-01.mp3  इन में 260 महिला सिपाही जिबकि 40 पुरुष सिपाही हैं 31 अगस्त को 260 महिला स

KeyboardInterrupt: 

In [13]:
pip install asr-evaluation

In [14]:
def wer(reference, hypothesis):
    reference_words = reference.split()
    hypothesis_words = hypothesis.split()
    dp = [[0] * (len(hypothesis_words) + 1) for _ in range(len(reference_words) + 1)]
    for i in range(len(reference_words) + 1):
        dp[i][0] = i
    for j in range(len(hypothesis_words) + 1):
        dp[0][j] = j

    for i in range(1, len(reference_words) + 1):
        for j in range(1, len(hypothesis_words) + 1):
            if reference_words[i - 1] == hypothesis_words[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = min(dp[i - 1][j - 1], dp[i - 1][j], dp[i][j - 1]) + 1

    return dp[len(reference_words)][len(hypothesis_words)] / len(reference_words)


reference_text = "मैं भी अच्छा हूँ"
hypothesis_text = "मैं बहुत अच्छा हूँ"
wer_score = wer(reference_text, hypothesis_text)
print("WER:", wer_score)


WER: 0.25


In [15]:
import re

def wer(reference, hypothesis):
    reference_words = reference.split()
    hypothesis_words = hypothesis.split()

    dp = [[0] * (len(hypothesis_words) + 1) for _ in range(len(reference_words) + 1)]

    for i in range(len(reference_words) + 1):
        dp[i][0] = i
    for j in range(len(hypothesis_words) + 1):
        dp[0][j] = j

    for i in range(1, len(reference_words) + 1):
        for j in range(1, len(hypothesis_words) + 1):
            if reference_words[i - 1] == hypothesis_words[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = min(dp[i - 1][j - 1], dp[i - 1][j], dp[i][j - 1]) + 1

    return dp[len(reference_words)][len(hypothesis_words)] / len(reference_words)

def wer_from_files(reference_file, hypothesis_file):
    with open(reference_file, 'r', encoding='utf-8') as f:
        reference_text = f.read()
        pattern = r'[\u0900-\u097F\s]+'
        hindi_sentences = re.findall(pattern, reference_text)

    print("Reference Text:")
    for sentence in hindi_sentences:
        print(sentence)

    with open(hypothesis_file, 'r', encoding='utf-8') as f:
        hypothesis_text = f.read().split(':')[-1]

    total_words = sum(len(line.split()) for line in hindi_sentences)

    total_errors = sum(wer(reference_line, hypothesis_line) * len(reference_line.split()) for reference_line, hypothesis_line in zip(hindi_sentences, hypothesis_text))

    wer_score = total_errors / total_words

    print("\nWER:", wer_score)

reference_file_path = '/content/drive/MyDrive/GV_Eval_3h/text'
hypothesis_file_path = '/content/drive/MyDrive/GV_Eval_3h/Result_Text.txt'
wer_from_files(reference_file_path, hypothesis_file_path)


Reference Text:
 वहीं राहुल कुमार मुकेश कुमार अरुण कुमार राजा कुमार अनुराग कुमार सनी कुमार रहीश कुमार मनीष कुमार सहित दर्जनों कार्यकर्ता भी शामिल थे धन्यवाद दिलीप पांडे जमुई अघहारा से बिहार मोबइल वाणी न्यूज़ एक्सप्रेस से

 युवा की अहम् जिम्मेदारी है इसलिए बहुत समय या पैसे खर्च करने की ज़रुरत नहीं है  दिन

 क्यूँकि हमारी पुलिस तो फेसबुक और दुसरे सोशल मीडिया के माध्यमों पर अपराध करने वालों को पकड़ने में ही असमर्थ है

 नाली जाम है जिससे निकालता गन्दा पानी सड़को पर जमा है जिसकी सफाई को लेकर किसी प्रकार की कयावत होती भी नहीं दिखती है

 समस्या को सुना जायेगा और महिलाओं की समस्या महिलाओं समझ सकती है इस इस विचार को रखते हुए सरकार ने महिलाओं के प्र 

 विकास के लिए शराब बंदी एक बड़ा कदम है इस मुद्दे पर जदूय जि ज़िला अध्यक्ष शंकर चौधरी जदूय नेता

 इस मध्य प्रदेश मोबाइल वाणी के सभी सुनने वाले श्रोताओं को और मेरे मित्रों को सभी को हार्दिक हार्दिक शुभकामनाएं धन्यवाद नमस्कार

 स्तर गिरने का मुख्य कारण  संस्कृति और परम्पराओं का सबक सबक बच्चों को न पढ़ाना है दुर्भाग्य की बात है कि भारत मैं आज लोग शिक्षा को मा

In [1]:
import re
text="01-00008-03 युवा की अहम् जिम्मेदारी है इसलिए बहुत समय या पैसे खर्च करने की ज़रुरत नहीं है  दिन"

pattern = r'[\u0900-\u097F\s]+'

hindi_sentences = re.findall(pattern, text)

complete_hindi_sentence = ''.join(hindi_sentences)

print(complete_hindi_sentence)


 युवा की अहम् जिम्मेदारी है इसलिए बहुत समय या पैसे खर्च करने की ज़रुरत नहीं है  दिन
